In [2]:
import pandas as pd

In [5]:
# Load the data
train_sample_df = pd.read_csv('../data/sample/train_sample_unlabeled.csv', sep=";")

# read in data
task_df = pd.read_csv('../data/raw/tasks_sector_classified.csv', sep=";")
goals_df = pd.read_csv('../data/raw/goals_sdt_classified.csv', sep=";")

In [6]:
# build cross product of goals and task to get all unique combinations
all_items_df = goals_df.assign(key=1).merge(task_df.assign(key=1), on='key').drop('key', axis=1)

In [8]:
# create ids for not choosing same data as in the training set
train_sample_df['id'] = (train_sample_df['goal'] + 
                         train_sample_df['title'] + 
                         train_sample_df['description'])
all_items_df['id'] = (all_items_df['goal'] + 
                      all_items_df['title'] + 
                      all_items_df['description'])

In [9]:
# remove items that are in the training set
all_items_df = all_items_df[~all_items_df['id'].isin(train_sample_df['id'])]
all_items_df

,goal,sdt-cluster,title,description,sector,source,id
0,I would like to be popular and be in the cente...,relatedness,"Volunteering with children and youth at ""Elisa...",Activity description\n The Elisabethhei...,Kinderbetreuung,EU,I would like to be popular and be in the cente...
1,I would like to be popular and be in the cente...,relatedness,Volunteering in Sicily in a housing community ...,Activity description\n Istituto Walden ...,Soziales und Gesundheit,EU,I would like to be popular and be in the cente...
2,I would like to be popular and be in the cente...,relatedness,Volunteering in a community home shelter for k...,Activity description\n The following ma...,Kinderbetreuung,EU,I would like to be popular and be in the cente...
3,I would like to be popular and be in the cente...,relatedness,Educational accompaniment for children,Activity description\n Paz y Esperanza ...,Religion und Kirche,EU,I would like to be popular and be in the cente...
4,I would like to be popular and be in the cente...,relatedness,Development of educational methodologies and i...,Activity description\n Paz y Esperanza ...,Politische Arbeit und Interessensvertretung,EU,I would like to be popular and be in the cente...
...,...,...,...,...,...,...,...
505015,I would like to volunteer for 1 h each week,autonomy,Fussballtrainer gesucht,Unser langjähriger ehrenamtlicher Fußballtrain...,Sport und Bewegung,SELF,I would like to volunteer for 1 h each weekFus...
505016,I would like to volunteer for 1 h each week,autonomy,Fit im Alter (Gruppenkurs Bewegung Alte Menschen),Viele Alterserkrankungen lassen sich durch aus...,Sport und Bewegung,SELF,I would like to volunteer for 1 h each weekFit...
505017,I would like to volunteer for 1 h each week,autonomy,Schnalzerfest - Kinderbetreuung,Für unser Schnalzerfest suchen wir von 10:00 -...,"Kunst, Kultur, Unterhaltung",SELF,I would like to volunteer for 1 h each weekSch...
505018,I would like to volunteer for 1 h each week,autonomy,Schnalzen schnuppern / Einführung,Du wolltest schon immer mehr über die Kunst de...,"Kunst, Kultur, Unterhaltung",SELF,I would like to volunteer for 1 h each weekSch...


In [10]:
def sample_group(group, frac=0.00025):
    return group.sample(frac=frac)

In [12]:
test_sample_df = all_items_df.groupby(['sdt-cluster', 'sector', 'source']).apply(sample_group).reset_index(drop=True)

/var/folders/_z/23l2kzhs6r1dz6q78mm2q1lw0000gn/T/ipykernel_22518/3790222561.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_sample_df = all_items_df.groupby(['sdt-cluster', 'sector', 'source']).apply(sample_group).reset_index(drop=True)


In [13]:
test_sample_df.groupby(['sdt-cluster', 'sector', 'source']).size().reset_index(name='count')

,sdt-cluster,sector,source,count
0,autonomy,Bildung,EU,5
1,autonomy,Bildung,VC,1
2,autonomy,Bildung,VM,1
3,autonomy,Bürgerliche Aktivitäten und Gemeinwesen,VC,1
4,autonomy,Büro und Wirtschaft,VM,1
...,...,...,...,...
63,relatedness,Sport und Bewegung,EU,1
64,relatedness,Sport und Bewegung,VC,1
65,relatedness,"Umwelt, Natur, Tierschutz",EU,3
66,relatedness,"Umwelt, Natur, Tierschutz",VC,1


In [ ]:
# save the test sample
test_sample_df.to_csv('../data/sample/test_sample_unlabeled.csv', sep=";", index=False)